In [3]:
import datetime
import easygui
import cv2
import librosa
import numpy as np
from  matplotlib import pyplot as plt
# from vidgear.gears import VideoGear, WriteGear 
import math

In [3]:
INPUT_VIDEO = easygui.fileopenbox(msg='Choose VIDEO for input', default='/mnt/Double_Dragon/Videography/', filetypes=['*.MP4', '*.mp4' ])
INPUT_AUDIO = easygui.fileopenbox(msg='Choose AUDIO for input', default='/mnt/Double_Dragon/Videography/', filetypes=['*.MP3', '*.mp3', '*.aac' ])

print ('INPUT_VIDEO: ' + INPUT_VIDEO) if INPUT_VIDEO else print('NO INPUT_VIDEO SELECTED')
print('INPUT_AUDIO: ' + INPUT_AUDIO) if INPUT_AUDIO else print('NO INPUT_AUDIO SELECTED')

INPUT_VIDEO: /mnt/Double_Dragon/Videography/Archive/20221127_Χαϊδάρι_Βράδυ_Πάρκο/P1150201_20221127181010_ROT90.MP4
INPUT_AUDIO: /home/user7/Bitwig Studio/Projects/20221112_TimpArmy/20221114_TimpArmy_B 2022-11-28_0015_Bd.mp3


In [2]:
def rename_clip_AddTimestamp (_filename):
    timestamp = datetime.datetime.now()
    t = str(timestamp.year), str(timestamp.month), str(timestamp.day), str(timestamp.hour), str(timestamp.minute), str(timestamp.second)
    return _filename.replace('.MP4', '_' + ''.join(t) + '.MP4')

In [38]:
INPUT_VIDEO =  '/mnt/Double_Dragon/Videography/Archive/20221119_Χαϊδάρι_Νίκαια/P1150137_20221121203748_ROT90.MP4'
INPUT_AUDIO_01 = '/home/user7/Bitwig Studio/Projects/20220926_RandomOrgan_Glitch/20221201_RandomOrgan_DroneGlitch_BEAT.mp3'
INPUT_AUDIO_02 = '/home/user7/Bitwig Studio/Projects/20220926_RandomOrgan_Glitch/20221201_RandomOrgan_DroneGlitch_ALL.mp3'

In [39]:
_video = VideoGear(source=INPUT_VIDEO).start()
_video_CV = cv2.VideoCapture(INPUT_VIDEO)
amp01, samplerate01= librosa.load(path=INPUT_AUDIO_01, mono= True)
amp01 = np.abs(amp01)

output_params = {"-input_framerate": _video.framerate}

new_video_filename = rename_clip_AddTimestamp(INPUT_VIDEO)
writer = WriteGear(output_filename="output_video.MP4", **output_params)

audioframe_num = 0
while True:
    # frame= _video.read()
    fr_exists, frame_CV = _video_CV.read()
    if fr_exists:
            frame_timestamp = _video_CV.get(cv2.CAP_PROP_POS_MSEC) 
    else: 
        break
    
    # modify frame
    frame_amp = math.floor( amp01[audioframe_num]*150)
    if frame_amp > 1 :
        new_frame = cv2.blur(frame_CV,  (frame_amp, frame_amp))
    else:
        new_frame = frame_CV
    # print (str(frame_timestamp) + '\t\t\t\t' + str(amp01[audioframe_num])+ '\t\t\t\t' + str(audioframe_num))
    writer.write(new_frame)

    audioframe_num = librosa.time_to_samples(times=frame_timestamp*0.001, sr=samplerate01) 

_video.stop()
_video_CV.release
writer.close()

####################################################################################
# add output_audio.aac
ffmpeg_command = [
"-y",
"-i",
"output_video.MP4",
"-i",
INPUT_AUDIO_02,
"-c:v",
"copy",
"-c:a",
"copy",
"-map",
"0:v:0",
"-map",
"1:a:0",
"-shortest",
"output_video_AV.MP4"
]
writer.execute_ffmpeg_cmd(ffmpeg_command)

/home/user7/anaconda3/envs/AV_20210619/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [15]:
import cv2
import numpy as np

img = cv2.imread('mpv-shot0001.jpg')
rect = (50,50, img.shape[1]-50, img.shape[0]-50)
mask = np.zeros(img.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img2 = img*mask2[:,:,np.newaxis]
cv2.imshow('test', img2)

# cv2.imshow('Glitched2', cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT))
# img_canny = cv2.Canny(img, 20, 20)
cv2.imshow('Original', img)
# cv2.imshow('Glitched', img_canny)
# cv2.imshow('Blurred 5', cv2.blur(img, (1, 1)))
# cv2.imshow('Blurred 15', cv2.blur(img, (15, 15)))
# cv2.imshow('Blurred 25', cv2.blur(img, (25, 25)))
# cv2.imshow('Blurred 35', cv2.blur(img, (35, 35)))

cv2.waitKey(0) 
cv2.destroyAllWindows()